In [1]:
import sys
sys.path.insert(0, '../')
import gc
import logging
logging.basicConfig(level=logging.DEBUG)

from cryptotrader.envs.driver import TrainingEnvironment, make_env
from cryptotrader.agents import apriori
from cryptotrader.random_process import ConstrainedOrnsteinUhlenbeckProcess
from cryptotrader.envs.utils import  get_historical, get_dfs_from_db
from cryptotrader.utils import convert_to, array_normalize, convert_and_clean

import pandas as pd
import pymongo as pm
from bokeh.io import output_notebook
from jupyterthemes import jtplot
output_notebook()
jtplot.style()
%matplotlib inline
import os
import numpy as np
np.random.seed(42)

# %load_ext line_profiler

Loading BokehJS ...

In [2]:
seed = 42
outdir = './logs'
data_dir = './data/'
# files = []
# for file in os.listdir(data_dir):
#     files.append(data_dir + file)

# Env params
freq = 30
obs_steps = 300
tax = 0.0025
init_fiat = 100.0
init_crypto = 0.0
max_eval_steps = 30
eval_runs = 300
batch_size = 16
test = True

In [3]:
# client = pm.MongoClient(host='192.168.1.100', port=27017, connect=True)
# db = client.db

# env = Apocalipse(name='poloniex_momentum_outofsample', seed=seed)
# # Set environment options
# env.set_freq(freq)
# env.set_obs_steps(obs_steps)

# symbols, dfs = get_dfs_from_db(db, 'poloniex', start='2017-02-00 00:00:00', end='2017-05-00 00:00:00', freq=str(freq) + 'min')  

# # Add backtest data
# for i, symbol in enumerate(symbols):
#     for col in dfs[i].columns:
#         dfs[i][col] = convert_and_clean(dfs[i][col])
#     env.add_df(df=dfs[i], symbol=symbol)
#     env.add_symbol(symbol)
#     env.set_init_crypto(init_crypto, symbol)
#     env.set_tax(tax, symbol)
# # del dfs, symbols

# gc.collect()
# env.set_init_fiat(init_fiat)

# # Clean pools
# env._reset_status()
# env.clear_dfs()
# gc.collect()

# env.set_training_stage(False)
# env.set_observation_space()
# env.set_action_space()
# env.reset(reset_funds=True, reset_results=True, reset_global_step=True);

In [4]:
env = TrainingEnvironment(name='poloniex_momentum_backtest', seed=seed)
# Set environment options
env.set_freq(freq)
env.set_obs_steps(obs_steps)

symbols = ['usdtbtc', 'usdteth', 'usdtetc', 'usdtxrp', 'usdtltc']
for i, symbol in enumerate(symbols):
    df = pd.read_csv(data_dir + symbol + '_fev_may.csv')
    df['date'] = df.date.apply(pd.to_datetime)
    df = df.set_index('date')
    for col in df.columns:
        df[col] = convert_and_clean(df[col])
    env.add_df(df=df, symbol=symbol)
    env.add_symbol(symbol)
    env.set_init_crypto(init_crypto, symbol)
    env.set_tax(tax, symbol)

gc.collect()
env.set_init_fiat(init_fiat)

# Clean pools
env._reset_status()
env.clear_dfs()
gc.collect()

env.set_training_stage(False)
env.set_observation_space()
env.set_action_space()
env.reset(reset_funds=True, reset_results=True, reset_global_step=True);

INFO:poloniex_momentum_backtest:[Training Environment initialization]
Training Environment Initialized!

INFO:poloniex_momentum_backtest:[<function TrainingEnvironment.set_action_space at 0x7ff7c36a3d90>]
Setting environment with 6 symbols.



In [5]:
random_process = ConstrainedOrnsteinUhlenbeckProcess(theta=1.0, size=env.action_space.low.shape[0])
random_agent = apriori.DummyTrader(random_process=random_process)

env.reset(reset_funds=True, reset_results=True, reset_global_step=True)
random_agent.test(env, nb_episodes=1, action_repetition=1, callbacks=None,
            visualize=False, nb_max_episode_steps=None, nb_max_start_steps=0, start_step_policy=None, verbose=True)

env.plot_results(env._get_results());

################### > Portifolio Performance Analysis < ###################-1 days +23:59:58.493731    

Portifolio excess Sharpe:                 -0.037281
Portifolio / Benchmark Sharpe ratio:      0.742104 / 0.728645
Portifolio / Benchmark Omega ratio:       1.177306 / 1.232671
Portifolio / Benchmark max drawdown:      -1.292271 / -81196.215602


In [ ]:
ed_agent = apriori.EqualyDistributedTrader()
env.set_training_stage(False)
env.reset(reset_funds=True, reset_results=True, reset_global_step=True)
ed_agent.test(env, verbose=True)

env.plot_results(env._get_results());

################### > Portifolio Performance Analysis < ###################-1 days +23:59:58.493731   

Portifolio excess Sharpe:                 0.001303
Portifolio / Benchmark Sharpe ratio:      1.268320 / 0.728645
Portifolio / Benchmark Omega ratio:       1.315809 / 1.232671
Portifolio / Benchmark max drawdown:      -126.127423 / -81196.215602


In [ ]:
# Optimize model hiperparameters
momentum_agent = apriori.MomentumTrader()

env.set_training_stage(True)
env.reset(reset_funds=True, reset_results=True, reset_global_step=True)

opt_params, info = momentum_agent.fit(env, eval_runs, batch_size, action_repetition=1, callbacks=None, verbose=True,
            visualize=False, nb_max_start_steps=0, start_step_policy=None, log_interval=10000,
            nb_max_episode_steps=max_eval_steps, n_workers=4)
print("\nOptimum parameters:", opt_params)

env.set_training_stage(False)
env.reset(reset_funds=True, reset_global_step=True, reset_results=True)

# Previous parameters found
# {'ma1': 39, 'ma2': 270, 'std_span': 299, 'std_weight_down': 1.9648256750026283, 'std_weight_up': 2.1424218749999993}
# {'ma1': 3, 'ma2': 38, 'std_span': 174, 'std_weight_down': 1.4932605779664736, 'std_weight_up': 1.3837483170117275}
# {'ma1': 25, 'ma2': 122, 'std_span': 292, 'std_weight_down': 1.38345703125, 'std_weight_up': 1.2510827681890813}

# momentum_agent.set_params(**{'ma1': 38, 'ma2': 270, 'std_span': 292, 'std_weight_down': 1.38345703125, 'std_weight_up': 1.2510827681890813})
momentum_agent.test(env, nb_episodes=1, action_repetition=1, callbacks=None,
            visualize=False, nb_max_episode_steps=None, nb_max_start_steps=0, start_step_policy=None, verbose=True)

print("\nEnv params: \nInit fiat: {}\nInit crypto: {}\nFreq: {}".format(env.init_fiat, env.init_crypto, env.freq))
env.plot_results(env._get_results());

In [ ]:
# # Optimize model hiperparameters
mesa_agent = apriori.MesaMomentumTrader()

env.set_training_stage(True)
env.reset(reset_funds=True, reset_results=True, reset_global_step=True)

opt_params, info = mesa_agent.fit(env, eval_runs, action_repetition=1, callbacks=None, verbose=True,
            visualize=False, nb_max_start_steps=0, start_step_policy=None, log_interval=10000,
            nb_max_episode_steps=max_eval_steps)

print("\nOptimum parameters:", opt_params)
env.set_training_stage(False)
env.reset(reset_funds=True, reset_global_step=True, reset_results=True)

mesa_agent.test(env, nb_episodes=1, action_repetition=1, callbacks=None,
            visualize=False, nb_max_episode_steps=None, nb_max_start_steps=0, start_step_policy=None, verbose=True)

print("\nEnv params: \nInit fiat: {}\nInit crypto: {}\nFreq: {}".format(env.init_fiat, env.init_crypto, env.freq))
env.plot_results(env._get_results());

In [ ]:
env = TrainingEnvironment(name='poloniex_momentum_oos', seed=seed)
# Set environment options
env.set_freq(freq)
env.set_obs_steps(obs_steps)

symbols = ['usdtbtc', 'usdteth', 'usdtetc', 'usdtxrp', 'usdtltc']
for i, symbol in enumerate(symbols):
    df = pd.read_csv(data_dir + symbol + '_may.csv')
    df['date'] = df.date.apply(pd.to_datetime)
    df = df.set_index('date')
    for col in df.columns:
        df[col] = convert_and_clean(df[col])
    env.add_df(df=df, symbol=symbol)
    env.add_symbol(symbol)
    env.set_init_crypto(init_crypto, symbol)
    env.set_tax(tax, symbol)

gc.collect()
env.set_init_fiat(init_fiat)

# Clean pools
env._reset_status()
env.clear_dfs()
gc.collect()

env.set_training_stage(False)
env.set_observation_space()
env.set_action_space()
env.reset(reset_funds=True, reset_results=True, reset_global_step=True);

In [ ]:
momentum_agent.test(env, nb_episodes=1, action_repetition=1, callbacks=None,
            visualize=False, nb_max_episode_steps=None, nb_max_start_steps=0, start_step_policy=None, verbose=True)

print("\nEnv params: \nInit fiat: {}\nInit crypto: {}\nFreq: {}".format(env.init_fiat, env.init_crypto, env.freq))
env.plot_results(env._get_results());

In [ ]:
mesa_agent.test(env, nb_episodes=1, action_repetition=1, callbacks=None,
            visualize=False, nb_max_episode_steps=None, nb_max_start_steps=0, start_step_policy=None, verbose=True)

print("\nEnv params: \nInit fiat: {}\nInit crypto: {}\nFreq: {}".format(env.init_fiat, env.init_crypto, env.freq))
env.plot_results(env._get_results());